# ALLEN

## Librerías

In [ ]:
import numpy as np
import pandas as pd
import re
from datetime import datetime,date
import json


!pip install -U googlemaps

import googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.5.3-py3-none-any.whl size=38479 sha256=7806d270a0388f97f2dbc88d696f84c40ff5871a6b9ede33456ef790e107cce4
  Stored in directory: /root/.cache/pip/wheels/fa/1a/1c/cc0b8a1652a3f06aea586b0e4714a81bafed830513969baf92
Successfully built googlemaps
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


## Cargue inicial del csv

In [ ]:
hurtos = pd.read_csv('Raw_data/hurtos_2009-2012.csv', sep = ';', header = None)
hurtos.columns = ['fecha','hora','direccion','categoria'] #Agregamos nombres a columnas
hurtos['fecha'] = hurtos['fecha'] + '/' + hurtos['hora'] # juntamos fecha y hora
hurtos = hurtos.drop(columns= ['hora'])

#Cambiamos los tipos de variables
#datetime
hurtos['fecha'] = pd.to_datetime( hurtos['fecha'], errors = 'coerce', format = '%d/%m/%Y/%H:%M') #errors: cada que encuentre un formato erroneo crea un NaN format: formato de la fecha '%m/%d/%Y %H:%M:%S %p'
#categórica
hurtos['categoria'] = hurtos['categoria'].astype('category')

#ajuste de redacción en direcciones para mejorar resultados de búsqueda en maps
hurtos['direccion'] = hurtos.apply(lambda x : x['direccion'].replace(re.findall(r'#\d+\s+|#\d+\w+\s+|\w+\d+\w+,|#\s+\d+|#\d+|,',x['direccion'])[0],re.findall(r'#\d+\s+|#\d+\w+\s+|\w+\d+\w+,|#\s+\d+|#\d+|,',x['direccion'])[0].replace(" ","-")),axis=1)

hurtos

,fecha,direccion,categoria
0,2009-01-01 13:00:00,"CL1 #2-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS
1,2009-01-01 00:00:00,"CR28 #43-2 , Palmira - Valle del Cauca, Colombia",HURTOPERSONAS
2,2009-01-02 02:00:00,"CL21 #24-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS
3,2009-01-02 02:35:00,"CL47 #25-32, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS
4,2009-01-02 08:00:00,"CL54B #45-60, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS
...,...,...,...
5839,2012-12-06 07:30:00,"CL47C#13-59, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS
5840,2012-12-11 07:30:00,"CR44#40-1 , Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS
5841,2012-12-30 20:30:00,"CL42#8, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS
5842,2012-12-02 02:00:00,"CR28#27-48 , Palmira - Valle del Cauca, Colombia",HURTOENTIDADESCOMERCIALES


## Transformación de direcciones a coordenadas (Testing)

In [ ]:
prueba = hurtos[0:1]
#prueba = pd.DataFrame(data=prueba)
#prueba.columns = ['direccion']
prueba['direccion']

0    CL1 #2-2, Palmira - Valle del Cauca, Colombia
Name: direccion, dtype: object

## Geocoding de lista de prueba (Testing)

In [ ]:
with open('creds.json') as data_file:
   data = json.load(data_file)

token = data['token']

gmaps = googlemaps.Client(key=token)

#test de conexión
#geocode_result = gmaps.geocode(str(prueba['direccion']))

In [ ]:
subset = hurtos[0:10] #subset de testing

In [ ]:
consulta = []
for index,row in subset.iterrows():
    consulta.append(str(gmaps.geocode(str(row['direccion']))))
    #consulta.append(str(row['direccion']))

In [ ]:
consulta[0] # el valor 'vacio' hay que quitarlo posicion 7

"[{'address_components': [{'long_name': 'Palmira', 'short_name': 'Palmira', 'types': ['locality', 'political']}, {'long_name': 'Palmira', 'short_name': 'Palmira', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Valle del Cauca', 'short_name': 'Valle del Cauca', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Colombia', 'short_name': 'CO', 'types': ['country', 'political']}], 'formatted_address': 'Palmira, Valle del Cauca, Colombia', 'geometry': {'bounds': {'northeast': {'lat': 3.5615751, 'lng': -76.2672759}, 'southwest': {'lat': 3.502452, 'lng': -76.32357499999999}}, 'location': {'lat': 3.5379718, 'lng': -76.2971657}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 3.5615751, 'lng': -76.2672759}, 'southwest': {'lat': 3.502452, 'lng': -76.32357499999999}}}, 'partial_match': True, 'place_id': 'ChIJ7VzukugEOo4RepOnnYtiHWQ', 'types': ['locality', 'political']}]"

In [ ]:
'bounds': {
'northeast': {'lat': 3.5615751, 'lng': -76.2672759}, 
'southwest': {'lat': 3.502452, 'lng': -76.32357499999999}}
north_lat_boud

In [ ]:
#Actualizada la extracción de las coordenadas
latitudes = []
longitudes = []
for i in consulta:
    latitud = float(re.search(r"'location': {'lat':\s+(.*?),\s+'lng'",i).group(1))
    longitud= float(re.search(r"'location': {'lat':\s+\d+.\d+,\s+'lng': (.*?)}",i).group(1))
    latitudes.append(latitud)
    longitudes.append(longitud)

NameError: name 'consulta' is not defined

In [ ]:
longitudes

[]

separar datos que no cargaron en un csv aparte para informar al usuario y trabajar en los modelos sin estos errores

algunos csv de entrada pueden contener las coordenadas, si estás se encuentran, no necesitamos transformar las direcciones.

## Geocoding del csv completo (Spoiler: Sale mal)

In [ ]:
#NO EJECUTAR, YA TENEMOS ALMACENADOS ESTOS DATOS
#consulta y extracción de direcciones geocodificadas
consulta = []
for index,row in hurtos.iterrows():
    consulta.append(str(gmaps.geocode(str(row['direccion']))))
    #consulta.append(str(row['direccion']))

consulta

KeyboardInterrupt: 

## Particionado (Chunking) del dataframe para las consultas con API Gmaps

💥Intentando hacer el geocoding al DataSet completo Deepnote se quedó sin memoria y reseteó el kernel, probaré separando por lotes el dataset total para evitar este inconveniente.

In [ ]:
n = round(len(hurtos)/5) #tamaño de cada chunk a partir de la cantidad de registros en el df
hurtos1 = hurtos[0*1:n*1]
hurtos2 = hurtos[n*1:n*2].reset_index()
hurtos3 = hurtos[n*2:n*3].reset_index()
hurtos4 = hurtos[n*3:n*4].reset_index()
hurtos5 = hurtos[n*4:n*5].reset_index()

In [ ]:
#validamos no tener perdida de registros
print("Cantidad de registros en lotes:",len(hurtos1)+len(hurtos2)+len(hurtos3)+len(hurtos4)+len(hurtos5),"\nCantidad de registros totales :",len(hurtos))

Cantidad de registros en lotes: 5844 
Cantidad de registros totales : 5844


In [ ]:
#NO EJECUTAR, YA TENEMOS ALMACENADOS ESTOS DATOS
#consulta y extracción de direcciones geocodificadas
consulta5 = []
for index,row in hurtos5.iterrows():
    consulta5.append(str(gmaps.geocode(str(row['direccion']))))

## Guardamos en csv

In [ ]:
#NO EJECUTAR, YA TENEMOS ESTOS DATOS
#se realiza la partición en 5 subsets para traer todos los datos de la api de google maps
convertidos = pd.DataFrame(consulta5)
convertidos.to_csv('Processed_data/convertido5.csv', index = False)

NameError: name 'consulta5' is not defined

## Traslado de los csv al Dataframe

In [ ]:
#primero los llamamos como dataframes
consulta1 = pd.read_csv("Processed_data/convertido1.csv", sep = ';')
consulta2 = pd.read_csv("Processed_data/convertido2.csv", sep = ';')
consulta3 = pd.read_csv("Processed_data/convertido3.csv", sep = ';')
consulta4 = pd.read_csv("Processed_data/convertido4.csv", sep = ';')
consulta5 = pd.read_csv("Processed_data/convertido5.csv", sep = ';')

## Extraemos resultados vacíos

In [ ]:
#crea una columna llamada vacíos, si el valor en dicha columna es 0,
#significa que encontró el registro vacío, de lo contrario dará -1

#dataframes generales
hurtos1['vacios'] = consulta1['0'].str.find('[]')
hurtos2['vacios'] = consulta2['0'].str.find('[]')
hurtos3['vacios'] = consulta3['0'].str.find('[]')
hurtos4['vacios'] = consulta4['0'].str.find('[]')
hurtos5['vacios'] = consulta5['0'].str.find('[]')

#coordenadas
consulta1['vacios'] = consulta1['0'].str.find('[]')
consulta2['vacios'] = consulta2['0'].str.find('[]')
consulta3['vacios'] = consulta3['0'].str.find('[]')
consulta4['vacios'] = consulta4['0'].str.find('[]')
consulta5['vacios'] = consulta5['0'].str.find('[]')

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
print("chunk 1:",len(hurtos1),len(consulta1),
      "\nchunk 2:",len(hurtos2),len(consulta2),
      "\nchunk 3:",len(hurtos3),len(consulta3),
      "\nchunk 4:",len(hurtos4),len(consulta4),
      "\nchunk 5:",len(hurtos5),len(consulta5))

chunk 1: 1169 1169 
chunk 2: 1169 1169 
chunk 3: 1169 1169 
chunk 4: 1169 1169 
chunk 5: 1168 1168


In [ ]:
#guardamos dataframe con errores 
errores_hurtos1 = hurtos1[hurtos1['vacios'] == 0] # 1 dirección no encontrada
errores_hurtos2 = hurtos2[hurtos2['vacios'] == 0] # sin errores
errores_hurtos3 = hurtos3[hurtos3['vacios'] == 0] # sin errores
errores_hurtos4 = hurtos4[hurtos4['vacios'] == 0] # sin errores
errores_hurtos5 = hurtos5[hurtos5['vacios'] == 0] # sin errores

errores_consulta1 = consulta1[consulta1['vacios'] == 0] # 1 dirección no encontrada
errores_consulta2 = consulta2[consulta2['vacios'] == 0] # sin errores
errores_consulta3 = consulta3[consulta3['vacios'] == 0] # sin errores
errores_consulta4 = consulta4[consulta4['vacios'] == 0] # sin errores
errores_consulta5 = consulta5[consulta5['vacios'] == 0] # sin errores

#dataframes generales sin errores
hurtos1  = hurtos1[hurtos1['vacios'] != 0]
hurtos2  = hurtos2[hurtos2['vacios'] != 0]
hurtos3  = hurtos3[hurtos3['vacios'] != 0]
hurtos4  = hurtos4[hurtos4['vacios'] != 0]
hurtos5  = hurtos5[hurtos5['vacios'] != 0]

#coordenadas sin errores
consulta1 = consulta1[consulta1['vacios'] != 0]
consulta2 = consulta2[consulta2['vacios'] != 0]
consulta3 = consulta3[consulta3['vacios'] != 0]
consulta4 = consulta4[consulta4['vacios'] != 0]
consulta5 = consulta5[consulta5['vacios'] != 0]

In [ ]:
print("chunk 1:",len(hurtos1),len(consulta1)  ,"Errores encontrados:",len(errores_consulta1),
      "\nchunk 2:",len(hurtos2),len(consulta2),"Errores encontrados:",len(errores_consulta2),
      "\nchunk 3:",len(hurtos3),len(consulta3),"Errores encontrados:",len(errores_consulta3),
      "\nchunk 4:",len(hurtos4),len(consulta4),"Errores encontrados:",len(errores_consulta4),
      "\nchunk 5:",len(hurtos5),len(consulta5),"Errores encontrados:",len(errores_consulta5))

chunk 1: 1168 1168 Errores encontrados: 1 
chunk 2: 1162 1162 Errores encontrados: 7 
chunk 3: 1163 1163 Errores encontrados: 6 
chunk 4: 1154 1154 Errores encontrados: 15 
chunk 5: 1164 1164 Errores encontrados: 4


In [ ]:
def extract_lat(consulta):
    lat_regex = r"'location': {'lat':\s+(.*?),\s+'lng'"
    latitudes = []
    
    for i in consulta['0']:
        latitudes.append(float(re.search(lat_regex,i).group(1)))
    return latitudes

def extract_lon(consulta):
    lon_regex = r"'location': {'lat':\s+\d+.\d+,\s+'lng': (.*?)}"
    longitudes = []

    for i in consulta['0']:
        longitudes.append(float(re.search(lon_regex,i).group(1)))
    return longitudes

In [ ]:
hurtos1['latitudes'] = extract_lat(consulta1)
hurtos2['latitudes'] = extract_lat(consulta2)
hurtos3['latitudes'] = extract_lat(consulta3)
hurtos4['latitudes'] = extract_lat(consulta4)
hurtos5['latitudes'] = extract_lat(consulta5)

In [ ]:
hurtos1['longitudes'] = extract_lon(consulta1)
hurtos2['longitudes'] = extract_lon(consulta2)
hurtos3['longitudes'] = extract_lon(consulta3)
hurtos4['longitudes'] = extract_lon(consulta4)
hurtos5['longitudes'] = extract_lon(consulta5)

In [ ]:
hurtos1

,fecha,direccion,categoria,vacios,latitudes,longitudes
0,2009-01-01 13:00:00,"CL1 #2-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.537972,-76.297166
1,2009-01-01 00:00:00,"CR28 #43-2 , Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.538685,-76.297222
2,2009-01-02 02:00:00,"CL21 #24-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.537972,-76.297166
3,2009-01-02 02:35:00,"CL47 #25-32, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.542123,-76.294796
4,2009-01-02 08:00:00,"CL54B #45-60, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.537972,-76.297166
...,...,...,...,...,...,...
1164,2009-09-21 16:00:00,"CL47 #34-2, Palmira - Valle del Cauca, Colombia",HURTOACOMERCIO,-1,3.542504,-76.302180
1165,2009-09-21 16:57:00,"CR28 #43-2, Palmira - Valle del Cauca, Colombia",HURTOACOMERCIO,-1,3.538685,-76.297222
1166,2009-09-22 16:00:00,"CR27 #30-31, Palmira - Valle del Cauca, Colombia",HURTOACOMERCIO,-1,3.527118,-76.298023
1167,2009-09-23 12:00:00,"CR32 #12-2, Palmira - Valle del Cauca, Colombia",HURTOACOMERCIO,-1,3.537972,-76.297166


In [ ]:
#hurtos_full = hurtos1.append(hurtos2,ignore_index=True)
hurtos_full = hurtos1.append(hurtos2)
hurtos_full = hurtos_full.append(hurtos3)
hurtos_full = hurtos_full.append(hurtos4)
hurtos_full = hurtos_full.append(hurtos5)
hurtos_full

,fecha,direccion,categoria,vacios,latitudes,longitudes,index
0,2009-01-01 13:00:00,"CL1 #2-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.537972,-76.297166,NaN
1,2009-01-01 00:00:00,"CR28 #43-2 , Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.538685,-76.297222,NaN
2,2009-01-02 02:00:00,"CL21 #24-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.537972,-76.297166,NaN
3,2009-01-02 02:35:00,"CL47 #25-32, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.542123,-76.294796,NaN
4,2009-01-02 08:00:00,"CL54B #45-60, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,-1,3.537972,-76.297166,NaN
...,...,...,...,...,...,...,...
1163,2012-12-06 07:30:00,"CL47C#13-59, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,-1,3.543396,-76.290278,5839.0
1164,2012-12-11 07:30:00,"CR44#40-1 , Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,-1,3.536834,-76.315036,5840.0
1165,2012-12-30 20:30:00,"CL42#8, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,-1,3.538710,-76.301890,5841.0
1166,2012-12-02 02:00:00,"CR28#27-48 , Palmira - Valle del Cauca, Colombia",HURTOENTIDADESCOMERCIALES,-1,3.524793,-76.299235,5842.0


In [ ]:
#dejamos el dataset completo y limpio de columnas innecesarias
hurtos_full = hurtos_full.drop(columns=['vacios','index'])
hurtos_full

,fecha,direccion,categoria,latitudes,longitudes
0,2009-01-01 13:00:00,"CL1 #2-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
1,2009-01-01 00:00:00,"CR28 #43-2 , Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.538685,-76.297222
2,2009-01-02 02:00:00,"CL21 #24-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
3,2009-01-02 02:35:00,"CL47 #25-32, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.542123,-76.294796
4,2009-01-02 08:00:00,"CL54B #45-60, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
...,...,...,...,...,...
1163,2012-12-06 07:30:00,"CL47C#13-59, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.543396,-76.290278
1164,2012-12-11 07:30:00,"CR44#40-1 , Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.536834,-76.315036
1165,2012-12-30 20:30:00,"CL42#8, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.538710,-76.301890
1166,2012-12-02 02:00:00,"CR28#27-48 , Palmira - Valle del Cauca, Colombia",HURTOENTIDADESCOMERCIALES,3.524793,-76.299235


🤔Revisando los datos completos se encuentra que hay direcciones que tienen coordenadas muy alejadas de la realidad, esto se debe a dos razones principalmente:
>*La dirección ingresada no tiene una ubicación exacta dentro de la ciudad entonces devuelve coordenadas alejadas.
>*La dirección ingresada coincide con otro lugar completamente diferente al que necesitamos

Todo esto sucede a pesar de que se precise la ciudad, departamento y país, al no encontrar coincidencias exactas, se va por una coincidencia similar en otros lugares.

In [ ]:
#filtrando datos erroneos
hurtos_filtrados = hurtos_full[(hurtos_full['latitudes'] <= hurtos_full['latitudes'].mean() + hurtos_full['latitudes'].std()) & (hurtos_full['latitudes'] >= hurtos_full['latitudes'].mean() - hurtos_full['latitudes'].std())]
hurtos_filtrados = hurtos_filtrados.reset_index().drop(columns='index')

In [ ]:
#Comprobamos los datos filtrados
#hurtos_filtrados.sort_values(by='latitudes')
#Solamente fue necesario filtrar la latitud para retirar las coincidencias impresisas de latitudes también

In [ ]:
hurtos_filtrados

,fecha,direccion,categoria,latitudes,longitudes
0,2009-01-01 13:00:00,"CL1 #2-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
1,2009-01-01 00:00:00,"CR28 #43-2 , Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.538685,-76.297222
2,2009-01-02 02:00:00,"CL21 #24-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
3,2009-01-02 02:35:00,"CL47 #25-32, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.542123,-76.294796
4,2009-01-02 08:00:00,"CL54B #45-60, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
...,...,...,...,...,...
5776,2012-12-06 07:30:00,"CL47C#13-59, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.543396,-76.290278
5777,2012-12-11 07:30:00,"CR44#40-1 , Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.536834,-76.315036
5778,2012-12-30 20:30:00,"CL42#8, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.538710,-76.301890
5779,2012-12-02 02:00:00,"CR28#27-48 , Palmira - Valle del Cauca, Colombia",HURTOENTIDADESCOMERCIALES,3.524793,-76.299235


In [ ]:
hurtos_filtrados.iloc[4338]

fecha                                      2002-01-15 06:00:00
direccion     CL43 #30-64, Palmira - Valle del Cauca, Colombia
categoria                            HURTOENTIDADESCOMERCIALES
latitudes                                             3.538958
longitudes                                          -76.299169
Name: 4338, dtype: object

## Corrección de datos con errores de redacción

In [ ]:
#hurtos_filtrados['fecha'].iloc[4338] = pd.Timestamp('2012-01-15T06')
#4239 4243 4251 4259 4267
hurtos_filtrados.iloc[4239]

fecha                                                  NaT
direccion     CR28#46, Palmira - Valle del Cauca, Colombia
categoria                                 HURTOAUTOMOTORES
latitudes                                         3.541816
longitudes                                      -76.296483
Name: 4239, dtype: object

In [ ]:
len(hurtos)-len(hurtos_filtrados)

63

In [ ]:
hurtos_filtrados.iloc[4239]

fecha                                                  NaT
direccion     CR28#46, Palmira - Valle del Cauca, Colombia
categoria                                 HURTOAUTOMOTORES
latitudes                                         3.541816
longitudes                                      -76.296483
Name: 4239, dtype: object

In [ ]:
hurtos[hurtos['direccion'] == "CR28#46, Palmira - Valle del Cauca, Colombia"] #20:30
hurtos[hurtos['direccion'] == "CL24#27, Palmira - Valle del Cauca, Colombia"] #hora 22:00
hurtos[hurtos['direccion'] == "CL35#13, Palmira - Valle del Cauca, Colombia"] #06:30

,fecha,direccion,categoria
3544,2011-05-27 21:20:00,"CL35#13, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS
4294,NaT,"CL35#13, Palmira - Valle del Cauca, Colombia",HURTOAUTOMOTORES


In [ ]:
hurtos[hurtos['direccion'] == "CL35#13-74, Palmira - Valle del Cauca, Colombia"] #06:30

,fecha,direccion,categoria
4302,NaT,"CL35#13-74, Palmira - Valle del Cauca, Colombia",HURTOAUTOMOTORES
4310,NaT,"CL35#13-74, Palmira - Valle del Cauca, Colombia",HURTOAUTOMOTORES
4318,NaT,"CL35#13-74, Palmira - Valle del Cauca, Colombia",HURTOAUTOMOTORES


Afortunadamente todos estos seis registros corresponden a una misma fecha 06-may-2011

In [ ]:
hurtos_filtrados[hurtos_filtrados['direccion'] == "CR28#46, Palmira - Valle del Cauca, Colombia"] #20:30
hurtos_filtrados['fecha'].iloc[4239] = pd.Timestamp('2011-05-06T2030')
hurtos_filtrados['fecha'].iloc[4239]

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Timestamp('2011-05-06 20:30:00')

In [ ]:
hurtos_filtrados[hurtos_filtrados['direccion'] == "CL24#27, Palmira - Valle del Cauca, Colombia"] #hora 22:00
hurtos_filtrados['fecha'].iloc[4232] = pd.Timestamp('2011-05-06T22')
hurtos_filtrados['fecha'].iloc[4232]

Timestamp('2011-05-06 22:00:00')

In [ ]:
hurtos_filtrados[hurtos_filtrados['direccion'] == "CL35#13, Palmira - Valle del Cauca, Colombia"] #06:30
hurtos_filtrados['fecha'].iloc[4243] = pd.Timestamp('2011-05-06T0630')
hurtos_filtrados.iloc[4243]

fecha                                  2011-05-06 06:30:00
direccion     CL35#13, Palmira - Valle del Cauca, Colombia
categoria                                 HURTOAUTOMOTORES
latitudes                                          3.53158
longitudes                                      -76.286017
Name: 4243, dtype: object

In [ ]:
hurtos_filtrados[hurtos_filtrados['direccion'] == "CL35#13-74, Palmira - Valle del Cauca, Colombia"] #06:30
hurtos_filtrados['fecha'].iloc[4251] = pd.Timestamp('2011-05-06T0630')
hurtos_filtrados['fecha'].iloc[4259] = pd.Timestamp('2011-05-06T0630')
hurtos_filtrados['fecha'].iloc[4267] = pd.Timestamp('2011-05-06T0630')

In [ ]:
hurtos_filtrados['fecha'].iloc[3789] = pd.Timestamp('2011-02-24T21') #otro adicional por corregir

## Exportación de datos limpios definitivos

In [ ]:
#Datos finalmente filtrados y listos
hurtos_filtrados

,fecha,direccion,categoria,latitudes,longitudes
0,2009-01-01 13:00:00,"CL1 #2-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
1,2009-01-01 00:00:00,"CR28 #43-2 , Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.538685,-76.297222
2,2009-01-02 02:00:00,"CL21 #24-2, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
3,2009-01-02 02:35:00,"CL47 #25-32, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.542123,-76.294796
4,2009-01-02 08:00:00,"CL54B #45-60, Palmira - Valle del Cauca, Colombia",HURTOPERSONAS,3.537972,-76.297166
...,...,...,...,...,...
5776,2012-12-06 07:30:00,"CL47C#13-59, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.543396,-76.290278
5777,2012-12-11 07:30:00,"CR44#40-1 , Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.536834,-76.315036
5778,2012-12-30 20:30:00,"CL42#8, Palmira - Valle del Cauca, Colombia",HURTOMOTOCICLETAS,3.538710,-76.301890
5779,2012-12-02 02:00:00,"CR28#27-48 , Palmira - Valle del Cauca, Colombia",HURTOENTIDADESCOMERCIALES,3.524793,-76.299235


In [ ]:
hurtos_filtrados.to_csv('Cleaned_data/hurtos0912Clean.csv', index = False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d4d7d92b-17a2-43d4-90a9-6387ff9a9276' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>